# Init Thingspeak into Kafka

In [2]:
import requests
from kafka import KafkaProducer
import json

def send_thingspeak_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from ThingSpeak
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        feeds = data.get("feeds", [])

        # Publish each entry to Kafka
        for feed in feeds:
            for field in fields:
                feed[fields[field]] = feed.pop(field)
            producer.send(topic, value=feed)
            #print(f"Sent: {feed}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(feeds)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

## Initiate beginning

In [9]:
from datetime import datetime, timedelta

# https://thingspeak.mathworks.com/channels/1785844 : Wind Power Smart Monitor
fields = {
    "field1": "wind_speed",
    "field2": "wind_power_density",
    "field3": "wind_power",
    "field4": "wind_air_density",
    "field5": "wind_temperature",
    "field6": "wind_pressure",
}

api_url = "https://thingspeak.mathworks.com/channels/1785844/feed.json?start={}&end={}"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

## Exemple with 10 calls.

In [10]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_thingspeak_to_kafka("wind", api_url.format(start, end), fields)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)
